https://www.youtube.com/watch?v=ztBJqzBU5kc

In [16]:

vector_db_array = []

# Functions

In [1]:
import glob

In [2]:
path = './docs/**/*.pdf'
file_list = glob.glob(path, recursive=True)
print(file_list)

['./docs/PZO2033E Interactive Maps.pdf', './docs/PZO2033 000 Cover.pdf', './docs/PZO2033 001-019.pdf', './docs/PZO2104E.pdf', './docs/PZO2101E-4th Printing.pdf', './docs/PZO1041E.pdf', './docs/PZO2102E.pdf']


In [3]:
embed_model="nomic-embed-text"

In [4]:
def load_data(file_path):
    # Load data from a file
    loader = UnstructuredPDFLoader(file_path=file_path)
    data = loader.load()
    return data

In [5]:
def new_file():
    f = open("answer.txt", "w")
    f.close()

In [6]:
new_file()

In [7]:
def print_result(question, chain):
    f = open("answer.txt", "a")
    f.write(f"Question: {question}\n")
    print(f"Question: {question}\n")
    f.write("==============================\n")
    print("==============================\n")
    answer = chain.invoke(question)
    f.write(f"{answer}\n")
    f.write("==============================\n\n")
    print(answer)

In [8]:
def data_reset(database):
    # Delete all collections in the db
    database.delete_collection()
    # vector_db.delete_collection()


# Ingest PDF

In [9]:
# !pip install --q unstructured langchain
# !pip install --"unstructured[all-docs]"
!poetry add unstructured langchain
!poetry add "unstructured[all-docs]"

The following packages are already present in the pyproject.toml and will be skipped:

  - unstructured
  - langchain

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
Using version ^0.13.2 for unstructured

Updating dependencies
Resolving dependencies... (0.4s)

No dependencies to install or update


In [10]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [11]:
!ollama pull {embed_model} 

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕███████████

In [12]:
!ollama list

NAME                    	ID          	SIZE  	MODIFIED      
codellama:34b           	685be00e1532	19 GB 	5 days ago   	
llama2:latest           	78e26419b446	3.8 GB	2 weeks ago  	
llama2-uncensored:latest	44040b922233	3.8 GB	12 days ago  	
mistral:latest          	61e88e884507	4.1 GB	2 weeks ago  	
mixtral:latest          	7708c059a8bb	26 GB 	5 days ago   	
nomic-embed-text:latest 	0a109f422b47	274 MB	2 seconds ago	
openhermes:latest       	95477a2659b7	4.1 GB	11 days ago  	


In [13]:
# !pip install --q chromadb
# !pip install --q langchain-text-splitters
!poetry add chromadb langchain-text-splitters

The following packages are already present in the pyproject.toml and will be skipped:

  - chromadb
  - langchain-text-splitters

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [14]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [15]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)


# Define a file list

# Back to the show

In [17]:

for (i, file) in enumerate(file_list):
  data = load_data(file)
  chunks = text_splitter.split_documents(data)
  vector_db_array.append(
    Chroma.from_documents(
      # documents=data, 
      documents=chunks, 
      embedding=OllamaEmbeddings(model=f"{embed_model}",show_progress=True),
      collection_name=f"local-rag-{i}"
    )
  )
  #loader = UnstructuredPDFLoader(file_path=file)
  #data = loader.load()
  #print(f"File {i+1} loaded")

OllamaEmbeddings: 100%|██████████| 194/194 [01:26<00:00,  2.25it/s]


## Retrieval

In [18]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [19]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines. 
    Provide page numbers and full stats whenever possible.
    Original question: {question}""",
)

In [20]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are the dungeon master, and I am here to play a role playing game. 
    Your task is to manage the story as described in `PZO2033*.pdf` and to help 
    guide me through the campaign to reach the end of `PZO2033 001-019.pdf`. Do your best to generate
    five different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Roll a 1d20 (random 1-20), to determine stats for attacks and defense. 
    Provide page numbers and full stats whenever possible.
    Original question: {question}""",
)

In [21]:

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [22]:
from langchain.retrievers import EnsembleRetriever

In [23]:
retrievers = []
for db in vector_db_array:
  print(f"Adding retriever for {db}")
  retriever = MultiQueryRetriever.from_llm(db.as_retriever(), llm, prompt=QUERY_PROMPT)
  retrievers.append(retriever)



Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x12e3e3a50>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x30ba5c190>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x30f6e5050>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x310bdbe90>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x315fd0390>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x316a49410>
Adding retriever for <langchain_community.vectorstores.chroma.Chroma object at 0x319da1710>


In [24]:
retriever = EnsembleRetriever(retrievers=retrievers)

In [25]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
print_result("What is the story about?", chain)

Question: What is the story about?




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 55.40it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 47.43it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 55.15it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 53.79it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 82.37it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 The text appears to be excerpts from a rulebook for the tabletop role-playing game "Pathfinder." It includes information about various types of shields, as well as rules and guidelines for using weapons in the game. There doesn't seem to be an overarching story present in these particular passages. The text is written in a formal, instructional tone and appears to be intended to help players understand and apply the rules related to shields and weapons in Pathfinder.


In [27]:
print_result("Tell me the campaign overview in PZO2033 001-019.pdf", chain)

Question: Tell me the campaign overview in PZO2033 001-019.pdf




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 42.32it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 49.45it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 50.41it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 46.72it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 68.43it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 The document "PZO2033 001-019.pdf" appears to be a series of adventure hooks and encounters for the Pathfinder Roleplaying Game (RPG) designed for use in a campaign setting. Here's an overview of each page based on their titles:

1. "The Lost Temple of the Serpent Queen": The players hear rumors of a lost temple dedicated to a powerful serpent deity and set out to find it, facing traps and puzzles along the way.
2. "The Haunted Caravan": A caravan is attacked by undead, and the players are hired to investigate and put an end to the curse plaguing it.
3. "The Cursed Witch": The players must determine whether a local witch is truly cursed or a danger to the community and deal with the consequences of their actions.
4. "The Shape-shifting Menace": The players encounter a shape-shifting creature terrorizing a nearby village and investigate its origins, discovering a larger conspiracy.
5. "The Haunting of Harrowstone": The players are summoned to investigate the supernatural events occurri

In [28]:
print_result("I would like to investigate the cursed witch, and learn if they are truly cursed.", chain)

Question: I would like to investigate the cursed witch, and learn if they are truly cursed.




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 66.55it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 36.44it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 79.38it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 34.69it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 75.98it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 To investigate the cursed witch, you will need to gather information about her background, her alleged curse, and any potential leads or clues related to her supposed curse. Here's a suggested approach:
1. Gather Information: Use your contacts in the local community, especially those with knowledge of the supernatural, to learn what is known about the cursed witch. Ask around for rumors, legends, or facts that might help you understand her situation. You may need to use Persuasion, Intimidation, or Deception skills to extract information from reluctant sources.
2. Interview Witnesses: Talk to people who have seen the witch or who live near her dwelling. Ask them about her behavior before and after the supposed curse took effect, any unusual occurrences they've noticed, and their personal beliefs regarding curses and witches. Use active listening skills and follow up questions to gather as much detail as possible.
3. Inspect the Witch's Dwelling: If it is safe to do so, visit the witch

In [30]:
print_result("I would like to gather information about the cursed witch in PZO2033 001-019.pdf, what do I need to do in order to accomplish this task?", chain)

Question: I would like to gather information about the cursed witch in PZO2033 001-019.pdf, what do I need to do in order to accomplish this task?




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 52.40it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 49.48it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 54.19it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 51.27it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 77.21it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 To gather information about the cursed witch in PZO2033 01-019.pdf, you can follow these steps:

1. Open the PDF file using a PDF reader software such as Adobe Acrobat, Foxit Reader, or any other compatible software.
2. Use the search function (Ctrl+F) to look for keywords related to the cursed witch. Some possible keywords could be "cursed witch," "witch," or specific names mentioned in the text.
3. Read through the text carefully, taking notes of any relevant information you find. This may include the witch's name, appearance, abilities, motives, and backstory.
4. Look for illustrations or diagrams that might depict the cursed witch or provide additional details about her.
5. Check if there are any sidebars, boxes, or special callouts in the text that contain useful information about the cursed witch.
6. Consult with other sources, such as rules books, supplements, or online resources, to gain a better understanding of the context and background of the cursed witch in Pathfinder lor

In [31]:
print_result("I would like to learn more about the cursed witch - I would like to ask around for rumors about the cursed witch.", chain)

Question: I would like to learn more about the cursed witch - I would like to ask around for rumors about the cursed witch.




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 41.53it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 36.56it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 45.60it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 44.50it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 75.75it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 To ask around for rumors about the cursed witch in your campaign, you can try the following steps:

1. Identify potential NPCs who might have information about the cursed witch: These could be local villagers, townsfolk, travelers, or even members of the clergy or other religious organizations.
2. Approach these NPCs and ask them if they have heard anything about a cursed witch in the area. Use your charisma skill to influence their response, and remember that they may not be forthcoming with information right away.
3. If an NPC does share some rumors or information about the cursed witch, listen carefully and try to glean as much detail as possible. You may need to follow up with additional questions to get a clearer picture of what you're dealing with.
4. Keep in mind that the rumors you hear may not be entirely accurate or truthful. Some NPCs might be spreading false information to scare off travelers or protect their own interests.
5. As you gather more information, try to piece t

In [32]:
print_result("Are there any legends about the cursed witch?", chain)

Question: Are there any legends about the cursed witch?




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 51.49it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 50.85it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 53.81it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 54.19it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 84.55it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 I couldn't find specific information about a "cursed witch" in the provided text. The text focuses on rules and guidelines for upgrading armor and weapons with runes, as well as descriptions of some property runes that can be added to these items. If you have more context or details about this "cursed witch," I would recommend checking other sources or consult with your game master for any potential legends or stories related to the character.


In [33]:
print_result("Are there any NPCs in the area that would know more information about the cursed witch?", chain)

Question: Are there any NPCs in the area that would know more information about the cursed witch?




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.69it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 48.46it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 51.95it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 47.45it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 47.17it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 The provided text does not contain any specific information about an NPC who knows more about the cursed witch. However, it does mention changelings, who can be hag offspring and could potentially have knowledge about a local hag coven or curse. It is recommended to consult the rest of the document or supplementary materials for further details on potential NPCs.


In [34]:
print_result("I would like to inspect the witches dwelling and search for any clues related to the curse or related to supernatural activity.", chain)

Question: I would like to inspect the witches dwelling and search for any clues related to the curse or related to supernatural activity.




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 48.58it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 43.03it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 49.61it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 50.55it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 86.78it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 To inspect the witch's dwelling and search for clues related to the curse or supernatural activity, you can follow these steps:

1. Approach the witch's dwelling with caution and make sure it is safe to enter. If necessary, consult with local authorities or seek their assistance if there are any potential risks involved.
2. Before entering the dwelling, take note of any visible signs that may indicate supernatural activity, such as strange symbols, unusual smells, or eerie sounds.
3. Once inside, examine each room carefully and systematically. Look for anything out of the ordinary, including hidden compartments, strange markings, odd objects, or anything that seems to be connected to the curse or the witch's powers.
4. Use your senses and skills to gather information about the dwelling and its contents. For example:
   * If you have the Perception skill, use it to search for hidden items or signs of supernatural activity.
   * If you have the Investigation skill, use it to identify my

In [29]:
print_result("Tell me about giant spiders in pathfinder", chain)

Question: Tell me about giant spiders in pathfinder




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 51.32it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 45.44it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 50.72it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 46.83it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 83.63it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings

 Giant spiders are a type of large arachnid creature found in the Pathfinder Roleplaying Game. They are often larger and more dangerous versions of their smaller counterparts, with some abilities that make them formidable opponents. Here's some basic information about giant spiders:

**Size:** Giant spiders are Large creatures, making them bigger than standard-sized spiders.

**Appearance:** Giant spiders have large, hairy bodies and eight long, spindly legs. Their fangs are capable of injecting a potent venom into their prey. They typically have a carapace that ranges in color from brown to black.

**Habitat:** Giant spiders can be found in various environments such as forests, deserts, caves, and even urban areas. Some species may prefer specific climates or regions.

**Behavior:** Giant spiders are typically solitary creatures but may form small colonies in large webs. They are ambush predators and will lie in wait for hours to catch their prey off guard. Once they sense vibrations,

In [82]:
# print_result("Tell me about giant spiders in pathfinder?", chain)

In [83]:
# print_result("How do you level a wizard in pathfinder?", chain)

In [84]:
# print_result("Do you know what a magus is in pathfinder?", chain)

In [85]:
# print_result("I have a ranged magus character with ranged spellstrike, what spells do you suggest that I take with my character into battle?", chain)

In [86]:
# print_result("What kind of spells should a level 1 cleric and level 10 magus prepare?", chain)

In [87]:
# print_result("Describe the spell snowball as a range spell", chain)


In [88]:
# print_result("What is the advantage to using acid arrow", chain)

In [89]:
# print_result("What kind of background should I give my new character in pathfinder?", chain)

In [90]:
# print_result("What are some things that could have happened in my early life in pathfinder", chain)

In [91]:
# print_result("What are some feats that you recommend for a level 10 megus with a level 1 dip into cleric ", chain)

In [94]:
# print_result("What are the gods in pathfinder?", chain)

In [95]:
# print_result("tell me more about the clerics of Desna", chain)

In [96]:
# print_result("What are the less popular gods in pathfinder?", chain)

In [97]:
# print_result("What are some gods that use whips as their preferred weapon?", chain)